In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

events = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
training = pd.read_csv('labels_training_set.csv', low_memory = False)
kaggle = pd.read_csv('trocafone_kaggle_test.csv', low_memory = False)

In [61]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [62]:
eventsWithLabel = events.merge(training, how='left', left_on='person', right_on='person')

In [63]:
train = eventsWithLabel.loc[eventsWithLabel["label"].notnull()]
test = eventsWithLabel.loc[eventsWithLabel["label"].isnull()] 
test = test.drop("label", axis=1)

In [64]:
kaggle = pd.merge(kaggle, test, how ="left", on = "person")

In [65]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
kaggle['timestamp'] = pd.to_datetime(kaggle['timestamp'])

/home/martin/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
diccWeek = {0: "Monday",
            1: "Tuesday",
            2: "Wednesday",
            3: "Thursday",
            4: "Friday",
            5: "Saturday",
            6: "Sunday"}

In [67]:
#Obtengo los dias de la semana
train['dayweek'] = train.timestamp.dt.dayofweek
train["dayweek"] = train["dayweek"].map(lambda x: diccWeek.get(x))
#Obtengo la fecha
train['fecha'] = train.timestamp.dt.date
#Separo la marca
train['marca'] = train['model'].map(lambda x: str(x).split()[0])

#Obtengo los dias de la semana
kaggle['dayweek'] = kaggle.timestamp.dt.dayofweek
kaggle["dayweek"] = kaggle["dayweek"].map(lambda x: diccWeek.get(x))
#Obtengo la fecha
kaggle['fecha'] = kaggle.timestamp.dt.date
#Separo la marca
kaggle['marca'] = kaggle['model'].map(lambda x: str(x).split()[0])

/home/martin/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/martin/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/martin/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [68]:
groupbyCantEventos = train.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos.sort_values(by = 'cantEventos', ascending = False)
train = pd.merge(train, groupbyCantEventos, on = 'person')

groupbyCantEventos2 = kaggle.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos2.sort_values(by = 'cantEventos', ascending = False)
kaggle = pd.merge(kaggle, groupbyCantEventos2, on = 'person')

In [69]:
train2 = train.loc[train['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

kaggle2 = kaggle.loc[kaggle['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

In [70]:
train3 = train.merge(train2, left_on = ['person', 'fecha'], right_on= ['person', 'fecha'], how='left')
train3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
train3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

kaggle3 = kaggle.merge(kaggle2, left_on = ['person', 'fecha'], right_on= ['person', 'fecha'], how='left')
kaggle3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
kaggle3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

In [71]:
train3["color"] = train3.color.str.split(" ", expand=True)
train3["SO"] = train3.SO.str.split(" ", expand=True)

kaggle3["color"] = kaggle3.color.str.split(" ", expand=True)
kaggle3["SO"] = kaggle3.SO.str.split(" ", expand=True)

In [72]:
dataCompraron = train3.loc[train3["label"] == 1]

In [73]:
dataCompraron["model"].value_counts()

iPhone 6                                       4182
iPhone 5s                                      4066
iPhone 6S                                      3595
Samsung Galaxy S7 Edge                         2548
iPhone 7                                       2353
Samsung Galaxy S7                              1790
Samsung Galaxy S6 Edge                         1725
Samsung Galaxy S6 Flat                         1565
Samsung Galaxy S8                              1418
iPhone 5c                                      1369
iPhone 7 Plus                                  1340
iPhone SE                                      1142
iPhone 6S Plus                                 1102
Samsung Galaxy J7 Prime                        1022
Samsung Galaxy J5                               916
Samsung Galaxy A7 2017                          888
iPhone 6 Plus                                   867
Motorola Moto G4 Plus                           864
Samsung Galaxy S5                               836
Samsung Gala

In [74]:
dataPersonRegion = train3.loc[train3["label"].notnull()].groupby("person")[["region", "label", "timestamp"]].agg("first")

In [75]:
dataRegiones = dataPersonRegion.groupby(["region", "label"]).count()
# dataRegionesFilter = dataPersonRegion.groupby(["region", "label" ])["timestamp"].count().unstack().fillna(value = 0)

dataRegionesFilter = dataRegiones.loc[dataRegiones["timestamp"] > 10]
dataRegionesFilter = dataRegionesFilter["timestamp"].unstack().fillna(value = 0)

In [76]:
dataRegionesFilter["ratioCompra"] = dataRegionesFilter[1.0]/dataRegionesFilter[0.0]

In [77]:
dataRegionesFilter.sort_values(by = "ratioCompra", ascending = False)

label,0.0,1.0,ratioCompra
region,,,
Santa Catarina,334.0,26.0,0.077844
Maranhao,317.0,24.0,0.075710
Para,291.0,22.0,0.075601
Rio Grande do Sul,539.0,36.0,0.066790
Minas Gerais,1487.0,88.0,0.059180
Parana,558.0,33.0,0.059140
Federal District,342.0,20.0,0.058480
Sao Paulo,5090.0,288.0,0.056582
Unknown,3099.0,168.0,0.054211


In [78]:
regionesMasCompras = ["Santa Catarina", "Maranhao", "Para", "Rio Grande do Sul", "Minas Gerais", "Parana", "Federal District", "Sao Paulo", "Espirito Santo", "Rio de Janeiro", "Bahia", "Goias", "Pernambuco"]

In [79]:
train3 = train3.drop(columns=["url", "sku", "skus", "staticpage", "marca", "campaign_source", "search_engine", "screen_resolution", "browser_version"])
kaggle3 = kaggle3.drop(columns=["url", "sku", "skus", "staticpage", "marca", "campaign_source", "search_engine", "screen_resolution", "browser_version"])

In [80]:
#Empezamos con las sesiones.
eventsWithSessionsTrain = train3.sort_values(by = ["person", "timestamp"])
eventsWithSessionsKaggle = kaggle3.sort_values(by = ["person", "timestamp"])

eventsWithSessionsTrain["time_diff"] = (eventsWithSessionsTrain.timestamp - (eventsWithSessionsTrain.timestamp.shift())) / np.timedelta64(1, 's')
eventsWithSessionsTrain.loc[eventsWithSessionsTrain.person != eventsWithSessionsTrain.person.shift(), "time_diff"] = 0

eventsWithSessionsKaggle["time_diff"] = (eventsWithSessionsKaggle.timestamp - (eventsWithSessionsKaggle.timestamp.shift())) / np.timedelta64(1, 's')
eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle.person != eventsWithSessionsKaggle.person.shift(), "time_diff"] = 0

In [81]:
eventsWithSessionsTrain["session_change"] = ((eventsWithSessionsTrain.event == "visited site") & (eventsWithSessionsTrain.time_diff > 1800.0)) | (eventsWithSessionsTrain.time_diff > 36000.0) | (eventsWithSessionsTrain.person != eventsWithSessionsTrain.person.shift())
eventsWithSessionsTrain["session_id"] = eventsWithSessionsTrain.groupby("person")["session_change"].cumsum()

eventsWithSessionsKaggle["session_change"] = ((eventsWithSessionsKaggle.event == "visited site") & (eventsWithSessionsKaggle.time_diff > 1800.0)) | (eventsWithSessionsKaggle.time_diff > 36000.0) | (eventsWithSessionsKaggle.person != eventsWithSessionsKaggle.person.shift())
eventsWithSessionsKaggle["session_id"] = eventsWithSessionsKaggle.groupby("person")["session_change"].cumsum()

In [82]:
dataSessionTrain =  eventsWithSessionsTrain.groupby(["person", "session_id"], as_index=False)["time_diff"].agg({"sessionDuration": "sum" })
dataSessionTrain["sessionDuration"] = dataSessionTrain["sessionDuration"]/60

dataSessionKaggle =  eventsWithSessionsKaggle.groupby(["person", "session_id"], as_index=False)["time_diff"].agg({"sessionDuration": "sum" })
dataSessionKaggle["sessionDuration"] = dataSessionKaggle["sessionDuration"]/60

In [83]:
dataSessionGroupByPersonTrain = dataSessionTrain.loc[dataSessionTrain.sessionDuration > 0.0].groupby("person", as_index = False)
dataSessionTrain = dataSessionGroupByPersonTrain.agg({"sessionDuration": {"sessionDuration": "mean"}, "session_id": {"cantSessions": "count"}})
dataSessionTrain.columns = ["person", "cantSessions", "sessionDuration"]

dataSessionGroupByPersonKaggle = dataSessionKaggle.loc[dataSessionKaggle.sessionDuration > 0.0].groupby("person", as_index = False)
dataSessionKaggle = dataSessionGroupByPersonKaggle.agg({"sessionDuration": {"sessionDuration": "mean"}, "session_id": {"cantSessions": "count"}})
dataSessionKaggle.columns = ["person", "cantSessions", "sessionDuration"]

/home/martin/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [84]:
eventsWithSessionsTrain = pd.merge(eventsWithSessionsTrain, dataSessionTrain, how ='left', on = "person")
eventsWithSessionsTrain["sessionDuration"] = eventsWithSessionsTrain["sessionDuration"].fillna(value = 0.0)
eventsWithSessionsTrain["cantSessions"] = eventsWithSessionsTrain["cantSessions"].fillna(value = 0.0)

eventsWithSessionsKaggle = pd.merge(eventsWithSessionsKaggle, dataSessionKaggle, how ='left', on = "person")
eventsWithSessionsKaggle["sessionDuration"] = eventsWithSessionsKaggle["sessionDuration"].fillna(value = 0.0)
eventsWithSessionsKaggle["cantSessions"] = eventsWithSessionsKaggle["cantSessions"].fillna(value = 0.0)

In [85]:
eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle['person'] == 'd21b8e6b']

,person,timestamp,event,model,condition,storage,color,search_term,dayweek,fecha,...,country,device_type,SO,channel,new_vs_returning,time_diff,session_change,session_id,cantSessions,sessionDuration
959461,d21b8e6b,2018-05-18 00:15:01,ad campaign hit,NaN,NaN,NaN,NaN,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,0.0,True,1.0,15.55,1.0
959462,d21b8e6b,2018-05-18 00:18:09,viewed product,Motorola Moto G3 4G,Bom,8GB,Preto,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,188.0,False,1.0,15.55,1.0
959463,d21b8e6b,2018-05-18 00:18:24,viewed product,Motorola Moto G3 4G,Bom,16GB,Preto,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,15.0,False,1.0,15.55,1.0
959464,d21b8e6b,2018-05-18 00:18:42,viewed product,Motorola Moto G3 4G,Bom,16GB,Branco,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,18.0,False,1.0,15.55,1.0
959465,d21b8e6b,2018-05-18 00:20:03,checkout,Motorola Moto G3 4G,Bom,16GB,Branco,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,81.0,False,1.0,15.55,1.0
959466,d21b8e6b,2018-05-18 00:26:01,ad campaign hit,NaN,NaN,NaN,NaN,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,358.0,False,1.0,15.55,1.0
959467,d21b8e6b,2018-05-18 00:26:07,generic listing,NaN,NaN,NaN,NaN,NaN,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,6.0,False,1.0,15.55,1.0
959468,d21b8e6b,2018-05-18 00:27:46,searched products,NaN,NaN,NaN,NaN,moto g5,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,99.0,False,1.0,15.55,1.0
959469,d21b8e6b,2018-05-18 00:28:14,searched products,NaN,NaN,NaN,NaN,moto g5,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,28.0,False,1.0,15.55,1.0
959470,d21b8e6b,2018-05-18 00:28:57,searched products,NaN,NaN,NaN,NaN,moto g3,Friday,2018-05-18,...,NaN,NaN,NaN,NaN,NaN,43.0,False,1.0,15.55,1.0


In [86]:
dataSearchSamsungTrain = eventsWithSessionsTrain.loc[eventsWithSessionsTrain['search_term'].notnull() & (eventsWithSessionsTrain['search_term'].str.contains('samsung') | eventsWithSessionsTrain['search_term'].str.contains('galaxy')),:].groupby('person')['search_term'].count().reset_index()
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(dataSearchSamsungTrain, how='left', left_on='person', right_on='person')
eventsWithSessionsTrain.rename(columns = {'search_term_x': 'search_term', 'search_term_y': 'samsung_searchs'}, inplace =True)

dataSearchSamsungKaggle = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle['search_term'].notnull() & (eventsWithSessionsKaggle['search_term'].str.contains('samsung') | eventsWithSessionsKaggle['search_term'].str.contains('galaxy')),:].groupby('person')['search_term'].count().reset_index()
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(dataSearchSamsungKaggle, how='left', left_on='person', right_on='person')
eventsWithSessionsKaggle.rename(columns = {'search_term_x': 'search_term', 'search_term_y': 'samsung_searchs'}, inplace =True)

In [87]:
dataSearchIphoneTrain = eventsWithSessionsTrain.loc[eventsWithSessionsTrain['search_term'].notnull() & eventsWithSessionsTrain['search_term'].str.contains('iphone'),:].groupby('person')['search_term'].count().reset_index()
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(dataSearchIphoneTrain, how='left', left_on='person', right_on='person')
eventsWithSessionsTrain.rename(columns = {'search_term_x': 'search_term', 'search_term_y': 'iphone_searchs'}, inplace =True)

dataSearchIphoneKaggle = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle['search_term'].notnull() & eventsWithSessionsKaggle['search_term'].str.contains('iphone'),:].groupby('person')['search_term'].count().reset_index()
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(dataSearchIphoneKaggle, how='left', left_on='person', right_on='person')
eventsWithSessionsKaggle.rename(columns = {'search_term_x': 'search_term', 'search_term_y': 'iphone_searchs'}, inplace =True)

In [88]:
eventsWithSessionsTrain['samsung_searchs'] = eventsWithSessionsTrain['samsung_searchs'].fillna(value=0)
eventsWithSessionsTrain['iphone_searchs'] = eventsWithSessionsTrain['iphone_searchs'].fillna(value=0)

eventsWithSessionsKaggle['samsung_searchs'] = eventsWithSessionsKaggle['samsung_searchs'].fillna(value=0)
eventsWithSessionsKaggle['iphone_searchs'] = eventsWithSessionsKaggle['iphone_searchs'].fillna(value=0)

In [89]:
def most_viewed(group):
    top_2_most_viewed = group.value_counts().head(2)
    if top_2_most_viewed.index[0] != "idk" or len(top_2_most_viewed) == 1:
        return top_2_most_viewed.index[0]
    else:
        return top_2_most_viewed.index[1]

In [90]:
eventsWithSessionsTrain['model'] = eventsWithSessionsTrain['model'].fillna(value='idk')
model_for_person = eventsWithSessionsTrain.groupby('person').model.apply(most_viewed).reset_index()
model_for_person = model_for_person.rename(columns=dict(level_2='model', model='most_viewed_model'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(model_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['model'] = eventsWithSessionsKaggle['model'].fillna(value='idk')
model_for_person2 = eventsWithSessionsKaggle.groupby('person').model.apply(most_viewed).reset_index()
model_for_person2 = model_for_person2.rename(columns=dict(level_2='model', model='most_viewed_model'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(model_for_person2, how='left', left_on='person', right_on='person')

In [91]:
eventsWithSessionsTrain['color'] = eventsWithSessionsTrain['color'].fillna(value='idk')
color_for_person = eventsWithSessionsTrain.groupby('person').color.apply(most_viewed).reset_index()
color_for_person = color_for_person.rename(columns=dict(level_2='color', model='most_viewed_color'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(color_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['color'] = eventsWithSessionsKaggle['color'].fillna(value='idk')
color_for_person2 = eventsWithSessionsKaggle.groupby('person').color.apply(most_viewed).reset_index()
color_for_person2 = color_for_person2.rename(columns=dict(level_2='color', model='most_viewed_color'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(color_for_person2, how='left', left_on='person', right_on='person')

In [92]:
eventsWithSessionsTrain['storage'] = eventsWithSessionsTrain['storage'].fillna(value='idk')
storage_for_person = eventsWithSessionsTrain.groupby('person').storage.apply(most_viewed).reset_index()
storage_for_person = storage_for_person.rename(columns=dict(level_2='storage', model='most_viewed_storage'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(storage_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['storage'] = eventsWithSessionsKaggle['storage'].fillna(value='idk')
storage_for_person2 = eventsWithSessionsKaggle.groupby('person').storage.apply(most_viewed).reset_index()
storage_for_person2 = storage_for_person2.rename(columns=dict(level_2='storage', model='most_viewed_storage'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(storage_for_person2, how='left', left_on='person', right_on='person')

In [93]:
eventsWithSessionsTrain['condition'] = eventsWithSessionsTrain['condition'].fillna(value='idk')
condition_for_person = eventsWithSessionsTrain.groupby('person').condition.apply(most_viewed).reset_index()
condition_for_person = condition_for_person.rename(columns=dict(level_2='condition', model='most_viewed_condition'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(condition_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['condition'] = eventsWithSessionsKaggle['condition'].fillna(value='idk')
condition_for_person2 = eventsWithSessionsKaggle.groupby('person').condition.apply(most_viewed).reset_index()
condition_for_person2 = condition_for_person2.rename(columns=dict(level_2='condition', model='most_viewed_condition'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(condition_for_person2, how='left', left_on='person', right_on='person')

In [94]:
eventsWithSessionsTrain['dayweek'] = eventsWithSessionsTrain['dayweek'].fillna(value='idk')
dayweek_for_person = eventsWithSessionsTrain.groupby('person').dayweek.apply(most_viewed).reset_index()
dayweek_for_person = dayweek_for_person.rename(columns=dict(level_2='dayweek', model='dayweek_most_events_done'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(dayweek_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['dayweek'] = eventsWithSessionsKaggle['dayweek'].fillna(value='idk')
dayweek_for_person2 = eventsWithSessionsKaggle.groupby('person').dayweek.apply(most_viewed).reset_index()
dayweek_for_person2 = dayweek_for_person2.rename(columns=dict(level_2='dayweek', model='dayweek_most_events_done'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(dayweek_for_person2, how='left', left_on='person', right_on='person')

In [95]:
eventsWithSessionsTrain['device_type'] = eventsWithSessionsTrain['device_type'].fillna(value='idk')
device_for_person = eventsWithSessionsTrain.groupby('person').device_type.apply(most_viewed).reset_index()
device_for_person = device_for_person.rename(columns=dict(level_2='device_type', model='most_used_device'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(device_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['device_type'] = eventsWithSessionsKaggle['device_type'].fillna(value='idk')
device_for_person2 = eventsWithSessionsKaggle.groupby('person').device_type.apply(most_viewed).reset_index()
device_for_person2 = device_for_person2.rename(columns=dict(level_2='device_type', model='most_used_device'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(device_for_person2, how='left', left_on='person', right_on='person')

In [96]:
eventsWithSessionsTrain['channel'] = eventsWithSessionsTrain['channel'].fillna(value='idk')
channel_for_person = eventsWithSessionsTrain.groupby('person').channel.apply(most_viewed).reset_index()
channel_for_person = channel_for_person.rename(columns=dict(level_2='channel', model='most_used_channel'))
eventsWithSessionsTrain = eventsWithSessionsTrain.merge(channel_for_person, how='left', left_on='person', right_on='person')

eventsWithSessionsKaggle['channel'] = eventsWithSessionsKaggle['channel'].fillna(value='idk')
channel_for_person2 = eventsWithSessionsKaggle.groupby('person').channel.apply(most_viewed).reset_index()
channel_for_person2 = channel_for_person2.rename(columns=dict(level_2='channel', model='most_used_channel'))
eventsWithSessionsKaggle = eventsWithSessionsKaggle.merge(channel_for_person2, how='left', left_on='person', right_on='person')

In [97]:
eventsWithSessionsTrain.drop(['model', 'search_term', 'channel_x', 'condition_x', 'storage_x', 'color_x', 'dayweek_x', 'device_type_x'], axis = 1, inplace = True)
eventsWithSessionsTrain.rename(columns = {'color_y': 'most_viewed_color', 'storage_y': 'most_viewed_storage', 'condition_y': 'most_viewed_condition', 'dayweek_y': 'dayweek_most_events_done', 'device_type_y': 'most_used_device', 'channel_y': 'most_used_channel'}, inplace =True)

eventsWithSessionsKaggle.drop(['model', 'search_term', 'channel_x', 'condition_x', 'storage_x', 'color_x', 'dayweek_x', 'device_type_x'], axis = 1, inplace = True)
eventsWithSessionsKaggle.rename(columns = {'color_y': 'most_viewed_color', 'storage_y': 'most_viewed_storage', 'condition_y': 'most_viewed_condition', 'dayweek_y': 'dayweek_most_events_done', 'device_type_y': 'most_used_device', 'channel_y': 'most_used_channel'}, inplace =True)

In [98]:
eventsWithSessionsTrain

,timestamp,event,person,label,fecha,cantEventos,city,region,country,SO,...,sessionDuration,samsung_searchs,iphone_searchs,most_viewed_model,most_viewed_color,most_viewed_storage,most_viewed_condition,dayweek_most_events_done,most_used_device,most_used_channel
0,2018-05-17 12:27:47,checkout,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
1,2018-05-17 13:44:59,visited site,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
2,2018-05-17 13:45:00,checkout,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
3,2018-05-17 16:21:54,visited site,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
4,2018-05-17 16:22:06,generic listing,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
5,2018-05-17 16:28:37,checkout,0008ed71,0.0,2018-05-17,6,Unknown,Unknown,Brazil,Windows,...,2.0,0.0,0.0,LG G4 H818P,Cinza,32GB,Bom,Thursday,Computer,Referral
6,2018-05-29 00:27:47,search engine hit,000c79fe,0.0,2018-05-29,17,Belo Horizonte,Minas Gerais,Brazil,Android,...,1.0,0.0,0.0,iPhone 7,Preto,128GB,Bom,Tuesday,Smartphone,Paid
7,2018-05-29 00:27:47,ad campaign hit,000c79fe,0.0,2018-05-29,17,Belo Horizonte,Minas Gerais,Brazil,Android,...,1.0,0.0,0.0,iPhone 7,Preto,128GB,Bom,Tuesday,Smartphone,Paid
8,2018-05-29 00:27:47,visited site,000c79fe,0.0,2018-05-29,17,Belo Horizonte,Minas Gerais,Brazil,Android,...,1.0,0.0,0.0,iPhone 7,Preto,128GB,Bom,Tuesday,Smartphone,Paid
9,2018-05-29 00:27:48,generic listing,000c79fe,0.0,2018-05-29,17,Belo Horizonte,Minas Gerais,Brazil,Android,...,1.0,0.0,0.0,iPhone 7,Preto,128GB,Bom,Tuesday,Smartphone,Paid


In [99]:
eventsWithSessionsKaggle

,person,timestamp,event,fecha,cantEventos,city,region,country,SO,new_vs_returning,...,sessionDuration,samsung_searchs,iphone_searchs,most_viewed_model,most_viewed_color,most_viewed_storage,most_viewed_condition,dayweek_most_events_done,most_used_device,most_used_channel
0,00091926,2018-05-03 22:08:29,visited site,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
1,00091926,2018-05-03 22:08:35,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
2,00091926,2018-05-03 22:08:51,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
3,00091926,2018-05-03 22:09:25,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
4,00091926,2018-05-03 22:09:34,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
5,00091926,2018-05-03 22:09:36,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
6,00091926,2018-05-03 22:09:59,viewed product,2018-05-03,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,New,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
7,00091926,2018-05-05 03:43:24,viewed product,2018-05-05,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,Returning,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
8,00091926,2018-05-05 03:43:24,visited site,2018-05-05,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,Returning,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct
9,00091926,2018-05-05 03:46:08,viewed product,2018-05-05,448,Carlos Barbosa,Rio Grande do Sul,Brazil,Windows,Returning,...,26.0,0.0,0.0,iPhone 6S,Preto,32GB,Muito Bom,Sunday,Computer,Direct


In [100]:
def LabelEncoding(column, uniqueArray, dataFrame):
    label_encoder = LabelEncoder()
    dataFrame[column] = label_encoder.fit_transform(dataFrame[column])

In [101]:
#Funcion que realiza el oneHotEncoding
def oneHotEncoding(column, uniqueArray, dataFrame):
    label_encoder = LabelEncoder()
    label_encoder.fit(uniqueArray)
    integer_encoded = label_encoder.transform(dataFrame[column])
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    for i, item in enumerate(uniqueArray):
        dataFrame[item] = onehot_encoded[:, label_encoder.transform(uniqueArray)[i]]

In [102]:
eventsColor = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_viewed_color"].notnull()][["most_viewed_color", "person"]]
eventsCondition = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_viewed_condition"].notnull()][["most_viewed_condition", "person"]]
eventsStorage = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_viewed_storage"].notnull()][["most_viewed_storage", "person"]]
eventsModel = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_viewed_model"].notnull()][["most_viewed_model", "person"]]
eventsChannel = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_used_channel"] != "Unknown"][["most_used_channel", "person"]]
eventsDevice = eventsWithSessionsTrain.loc[eventsWithSessionsTrain["most_used_device"] != "Unknown"][["most_used_device", "person"]]

eventsColor2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_viewed_color"].notnull()][["most_viewed_color", "person"]]
eventsCondition2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_viewed_condition"].notnull()][["most_viewed_condition", "person"]]
eventsStorage2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_viewed_storage"].notnull()][["most_viewed_storage", "person"]]
eventsModel2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_viewed_model"].notnull()][["most_viewed_model", "person"]]
eventsChannel2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_used_channel"] != "Unknown"][["most_used_channel", "person"]]
eventsDevice2 = eventsWithSessionsKaggle.loc[eventsWithSessionsKaggle["most_used_device"] != "Unknown"][["most_used_device", "person"]]

In [103]:
eventsWithSessionsTrain['SO'] = eventsWithSessionsTrain['SO'].fillna(value='idk')
eventsWithSessionsKaggle['SO'] = eventsWithSessionsKaggle['SO'].fillna(value='idk')

In [104]:
eventosUnicos = eventsWithSessionsTrain.drop_duplicates(subset="event", keep="first").reset_index()["event"]
coloresUnicos = eventsColor.drop_duplicates(subset='most_viewed_color', keep='first').reset_index()["most_viewed_color"]
conditionUnicos = eventsCondition.drop_duplicates(subset='most_viewed_condition', keep='first').reset_index()["most_viewed_condition"]
storageUnicos = eventsStorage.drop_duplicates(subset='most_viewed_storage', keep='first').reset_index()["most_viewed_storage"]
modelosUnicos = eventsModel.drop_duplicates(subset='most_viewed_model', keep='first').reset_index()['most_viewed_model']
channelUnicos = eventsChannel.drop_duplicates(subset="most_used_channel", keep="first").reset_index()["most_used_channel"]
SOUnicos = eventsWithSessionsTrain.drop_duplicates(subset="SO", keep="first").reset_index()["SO"]
deviceTypeUnicos = eventsDevice.drop_duplicates(subset="most_used_device", keep="first").reset_index()["most_used_device"]
daysWeekUnicos = eventsWithSessionsTrain.drop_duplicates(subset="dayweek_most_events_done", keep="first").reset_index()["dayweek_most_events_done"]

In [105]:
oneHotEncoding("event", eventosUnicos, eventsWithSessionsTrain)
LabelEncoding("most_viewed_color", coloresUnicos, eventsWithSessionsTrain)
LabelEncoding("most_viewed_condition", conditionUnicos, eventsWithSessionsTrain)
LabelEncoding("most_viewed_storage", storageUnicos, eventsWithSessionsTrain)
LabelEncoding("most_viewed_model", modelosUnicos, eventsWithSessionsTrain)
LabelEncoding("most_used_channel", channelUnicos, eventsWithSessionsTrain)
LabelEncoding("SO", SOUnicos, eventsWithSessionsTrain)
LabelEncoding("most_used_device", deviceTypeUnicos, eventsWithSessionsTrain)
LabelEncoding("dayweek_most_events_done", daysWeekUnicos, eventsWithSessionsTrain)

oneHotEncoding("event", eventosUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_viewed_color", coloresUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_viewed_condition", conditionUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_viewed_storage", storageUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_viewed_model", modelosUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_used_channel", channelUnicos, eventsWithSessionsKaggle)
LabelEncoding("SO", SOUnicos, eventsWithSessionsKaggle)
LabelEncoding("most_used_device", deviceTypeUnicos, eventsWithSessionsKaggle)
LabelEncoding("dayweek_most_events_done", daysWeekUnicos, eventsWithSessionsKaggle)

/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto

In [106]:
eventsWithSessionsKaggle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169795 entries, 0 to 1169794
Data columns (total 35 columns):
person                      1169795 non-null object
timestamp                   1169795 non-null datetime64[ns]
event                       1169795 non-null object
fecha                       1169795 non-null object
cantEventos                 1169795 non-null int64
city                        1158308 non-null object
region                      1158308 non-null object
country                     1158308 non-null object
SO                          1169795 non-null int64
new_vs_returning            1158308 non-null object
time_diff                   1169795 non-null float64
session_change              1169795 non-null bool
session_id                  1169795 non-null float64
cantSessions                1169795 non-null float64
sessionDuration             1169795 non-null float64
samsung_searchs             1169795 non-null float64
iphone_searchs              1169795 non-null f

In [107]:
eventsWithSessionsFilter = eventsWithSessionsTrain.groupby("person")[["sessionDuration", "cantSessions"]].agg("first")
eventsWithSessionsFilter2 = eventsWithSessionsKaggle.groupby("person")[["sessionDuration", "cantSessions"]].agg("first")

In [108]:
oneHotEvents = eventsWithSessionsTrain.loc[:,['person', 'searched products', 'viewed product', 'checkout', 'ad campaign hit', 'generic listing', 'brand listing', 'staticpage', 'search engine hit', 'conversion', 'lead']]
oneHotEvents2 = eventsWithSessionsKaggle.loc[:,['person', 'searched products', 'viewed product', 'checkout', 'ad campaign hit', 'generic listing', 'brand listing', 'staticpage', 'search engine hit', 'conversion', 'lead']]

In [109]:
oneHotSum = oneHotEvents.groupby("person").sum().reset_index()
oneHotSum2 = oneHotEvents2.groupby("person").sum().reset_index()

In [110]:
personModel = eventsWithSessionsTrain.loc[:,['person', 'most_viewed_model']].drop_duplicates('person')
personColor = eventsWithSessionsTrain.loc[:,['person', 'most_viewed_color']].drop_duplicates('person')
personStorage = eventsWithSessionsTrain.loc[:,['person', 'most_viewed_storage']].drop_duplicates('person')
personCondition = eventsWithSessionsTrain.loc[:,['person', 'most_viewed_condition']].drop_duplicates('person')
personChannel = eventsWithSessionsTrain.loc[:,['person', 'most_used_channel']].drop_duplicates('person')
personSO = eventsWithSessionsTrain.loc[:,['person', 'SO']].drop_duplicates('person')
personDevice = eventsWithSessionsTrain.loc[:,['person', 'most_used_device']].drop_duplicates('person')
personDayweek = eventsWithSessionsTrain.loc[:,['person', 'dayweek_most_events_done']].drop_duplicates('person')
personSamsungSearch = eventsWithSessionsTrain.loc[:,['person', 'samsung_searchs']].drop_duplicates('person')
personIphoneSearch = eventsWithSessionsTrain.loc[:,['person', 'iphone_searchs']].drop_duplicates('person')

personModel2 = eventsWithSessionsKaggle.loc[:,['person', 'most_viewed_model']].drop_duplicates('person')
personColor2 = eventsWithSessionsKaggle.loc[:,['person', 'most_viewed_color']].drop_duplicates('person')
personStorage2 = eventsWithSessionsKaggle.loc[:,['person', 'most_viewed_storage']].drop_duplicates('person')
personCondition2 = eventsWithSessionsKaggle.loc[:,['person', 'most_viewed_condition']].drop_duplicates('person')
personChannel2 = eventsWithSessionsKaggle.loc[:,['person', 'most_used_channel']].drop_duplicates('person')
personSO2 = eventsWithSessionsKaggle.loc[:,['person', 'SO']].drop_duplicates('person')
personDevice2 = eventsWithSessionsKaggle.loc[:,['person', 'most_used_device']].drop_duplicates('person')
personDayweek2 = eventsWithSessionsKaggle.loc[:,['person', 'dayweek_most_events_done']].drop_duplicates('person')
personSamsungSearch2 = eventsWithSessionsKaggle.loc[:,['person', 'samsung_searchs']].drop_duplicates('person')
personIphoneSearch2 = eventsWithSessionsKaggle.loc[:,['person', 'iphone_searchs']].drop_duplicates('person')

In [111]:
eventsWithSessionsKaggle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169795 entries, 0 to 1169794
Data columns (total 35 columns):
person                      1169795 non-null object
timestamp                   1169795 non-null datetime64[ns]
event                       1169795 non-null object
fecha                       1169795 non-null object
cantEventos                 1169795 non-null int64
city                        1158308 non-null object
region                      1158308 non-null object
country                     1158308 non-null object
SO                          1169795 non-null int64
new_vs_returning            1158308 non-null object
time_diff                   1169795 non-null float64
session_change              1169795 non-null bool
session_id                  1169795 non-null float64
cantSessions                1169795 non-null float64
sessionDuration             1169795 non-null float64
samsung_searchs             1169795 non-null float64
iphone_searchs              1169795 non-null f

In [112]:
kaggleDF = eventsWithSessionsKaggle.loc[:, ['person', 'timestamp']].drop_duplicates('person')
trainingDF = eventsWithSessionsTrain.loc[:, ['person', 'timestamp']].drop_duplicates('person')
labels = eventsWithSessionsTrain.loc[:, ['person', 'label']].drop_duplicates('person')

In [113]:
kaggleDF = pd.merge(kaggleDF, oneHotSum2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personModel2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personColor2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personStorage2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personCondition2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personChannel2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personSO2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personDevice2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personDayweek2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personIphoneSearch2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, personSamsungSearch2, how="left" , on = "person")
kaggleDF = pd.merge(kaggleDF, eventsWithSessionsFilter2, how="left" , on = "person")
kaggleDF = kaggleDF.drop('timestamp', 1)
kaggleDF

,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,conversion,...,most_viewed_storage,most_viewed_condition,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,sessionDuration,cantSessions
0,00091926,0.0,372.0,2.0,15.0,0.0,25.0,0.0,0.0,0.0,...,3,3,0,10,0,3,0.0,0.0,26.0,1545.521795
1,00091a7a,0.0,3.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,...,6,2,3,0,1,1,0.0,0.0,1.0,5.783333
2,000ba417,0.0,153.0,6.0,1.0,14.0,24.0,0.0,1.0,1.0,...,1,0,0,10,0,4,0.0,0.0,4.0,3269.404167
3,000e4d9e,0.0,339.0,1.0,19.0,17.0,17.0,0.0,5.0,0.0,...,3,3,2,10,0,6,0.0,0.0,10.0,2051.045000
4,000e619d,6.0,28.0,1.0,6.0,8.0,11.0,0.0,3.0,0.0,...,3,0,0,10,0,4,0.0,1.0,4.0,1051.375000
5,001001be,17.0,41.0,3.0,0.0,3.0,0.0,0.0,0.0,1.0,...,1,3,2,0,1,6,0.0,0.0,3.0,2762.488889
6,0010e89a,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7,0,3,10,0,6,0.0,0.0,1.0,0.716667
7,0016c4b5,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3,0,3,0,1,5,0.0,0.0,1.0,0.033333
8,001804a2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,4,12,0,1,0.0,0.0,2.0,1987.100000
9,001a2273,0.0,6.0,1.0,4.0,0.0,0.0,0.0,4.0,0.0,...,3,0,3,0,1,3,0.0,0.0,1.0,14.583333


In [114]:
trainingDF = pd.merge(trainingDF, oneHotSum, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personModel, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personColor, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personStorage, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personCondition, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personChannel, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personSO, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personDevice, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personDayweek, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personIphoneSearch, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, personSamsungSearch, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, eventsWithSessionsFilter, how="left" , on = "person")
trainingDF = pd.merge(trainingDF, labels, how="left" , on = "person")
trainingDF = trainingDF.drop('timestamp', 1)
trainingDF

,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,conversion,...,most_viewed_condition,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,sessionDuration,cantSessions,label
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,4,9,0,4,0.0,0.0,2.0,120.416667,0.0
1,000c79fe,9.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0,3,0,1,5,0.0,0.0,1.0,10.333333,0.0
2,001802e4,4.0,4.0,1.0,5.0,4.0,0.0,0.0,0.0,0.0,...,0,3,0,1,4,2.0,0.0,1.0,10.766667,0.0
3,0019e639,11.0,189.0,15.0,29.0,28.0,165.0,0.0,13.0,2.0,...,0,3,9,0,5,0.0,2.0,13.0,12663.842308,0.0
4,001b0bf9,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,...,2,7,11,4,5,0.0,0.0,1.0,15.300000,0.0
5,001ca5ee,0.0,52.0,1.0,7.0,8.0,7.0,0.0,6.0,0.0,...,2,3,0,1,6,0.0,0.0,14.0,8177.296429,0.0
6,001dfc31,0.0,18.0,0.0,5.0,2.0,0.0,0.0,6.0,0.0,...,2,3,0,1,5,0.0,0.0,4.0,36967.795833,0.0
7,001e9aea,0.0,17.0,3.0,13.0,0.0,4.0,0.0,10.0,0.0,...,0,3,0,1,3,0.0,0.0,6.0,16290.991667,0.0
8,0020152e,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,...,2,3,9,0,6,0.0,0.0,2.0,16880.241667,0.0
9,002b0188,0.0,12.0,1.0,2.0,3.0,17.0,0.0,1.0,0.0,...,0,2,9,0,4,0.0,0.0,2.0,44766.191667,0.0


In [118]:
#trainingDF.loc[trainingDF['person'] == 'd21b8e6b']
#kaggleDF.loc[kaggleDF['person'] == 'd21b8e6b']

,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,conversion,...,most_viewed_storage,most_viewed_condition,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,sessionDuration,cantSessions
15941,d21b8e6b,3.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1,0,7,12,4,0,0.0,0.0,1.0,15.55


In [115]:
#Creamos los csv ya listo para entrenar a los algoritmos y para testear que tan buenos son.
trainingDF.to_csv('setEntrenamientoMP.csv')
kaggle.to_csv('setKaggleMP.csv')